# 📘 1강 (CPU-Optimized, PyTorch): CNN 기초 — CIFAR-10

**GPU 없어도 실행 가능**합니다. 다만 학습 시간은 길어질 수 있으므로 아래 기본 설정(작은 배치/에폭)으로 구성했습니다.
- 기본: `epochs=8`, `batch_size=64`, `num_workers=0` (Windows/노트북 환경 호환성↑)
- 필요하면 `epochs`/`batch_size`를 조정하세요.


In [ ]:
import os, time, random, numpy as np, matplotlib.pyplot as plt
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.metrics import confusion_matrix, classification_report

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
set_seed(42)

In [ ]:
# 데이터셋/전처리
train_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2470,0.2435,0.2616))
])
test_tf = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2470,0.2435,0.2616))
])

root='./data'
train_full = datasets.CIFAR10(root=root, train=True, download=True, transform=train_tf)
test_set   = datasets.CIFAR10(root=root, train=False, download=True, transform=test_tf)
class_names = train_full.classes

# Train/Val split
val_ratio=0.2
val_len  = int(len(train_full)*val_ratio)
train_len= len(train_full)-val_len
train_set, val_set = random_split(train_full,[train_len,val_len])

batch_size=64
num_workers=0  # CPU 호환/안정성 위해 0 권장
train_loader=DataLoader(train_set,batch_size=batch_size,shuffle=True,num_workers=num_workers)
val_loader  =DataLoader(val_set,  batch_size=batch_size,shuffle=False,num_workers=num_workers)
test_loader =DataLoader(test_set,  batch_size=batch_size,shuffle=False,num_workers=num_workers)

len(train_set),len(val_set),len(test_set),class_names

In [ ]:
# 모델/학습 유틸
class CNN(nn.Module):
    def __init__(self,num_classes=10):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64,128,3,padding=1), nn.ReLU(),
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Dropout(0.3),
            nn.Linear(128,num_classes)
        )
    def forward(self,x):
        x=self.features(x); x=self.classifier(x); return x

def train_epoch(model,loader,loss_fn,optim_):
    model.train(); total_loss=0; correct=0; total=0
    for x,y in loader:
        x,y=x.to(device),y.to(device)
        optim_.zero_grad()
        logits=model(x); loss=loss_fn(logits,y)
        loss.backward(); optim_.step()
        total_loss+=loss.item()*x.size(0)
        correct+=(logits.argmax(1)==y).sum().item()
        total+=x.size(0)
    return total_loss/total, correct/total

@torch.no_grad()
def eval_epoch(model,loader,loss_fn):
    model.eval(); total_loss=0; correct=0; total=0
    for x,y in loader:
        x,y=x.to(device),y.to(device)
        logits=model(x); loss=loss_fn(logits,y)
        total_loss+=loss.item()*x.size(0)
        correct+=(logits.argmax(1)==y).sum().item()
        total+=x.size(0)
    return total_loss/total, correct/total

In [ ]:
# 학습
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

EPOCHS=8
hist={'train_acc':[],'val_acc':[],'train_loss':[],'val_loss':[]}
best_va=0.0; best_state=None

for ep in range(1,EPOCHS+1):
    tr_loss,tr_acc = train_epoch(model,train_loader,criterion,optimizer)
    va_loss,va_acc = eval_epoch(model,val_loader,criterion)
    hist['train_loss'].append(tr_loss); hist['val_loss'].append(va_loss)
    hist['train_acc'].append(tr_acc);   hist['val_acc'].append(va_acc)
    print(f'[Ep {ep}/{EPOCHS}] train_acc={tr_acc:.3f} val_acc={va_acc:.3f}')
    if va_acc>best_va: best_va=va_acc; best_state=model.state_dict().copy()
if best_state: model.load_state_dict(best_state)

In [ ]:
# 학습 곡선
import matplotlib.pyplot as plt
def plot_hist(h):
    plt.figure(figsize=(6,4))
    plt.plot(h['train_acc'],label='train_acc'); plt.plot(h['val_acc'],label='val_acc')
    plt.title('Accuracy'); plt.legend(); plt.tight_layout(); plt.show()
    plt.figure(figsize=(6,4))
    plt.plot(h['train_loss'],label='train_loss'); plt.plot(h['val_loss'],label='val_loss')
    plt.title('Loss'); plt.legend(); plt.tight_layout(); plt.show()
plot_hist(hist)

In [ ]:
# 테스트 평가 + 혼동행렬
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

@torch.no_grad()
def preds_and_labels(model, loader):
    model.eval(); ys=[]; ps=[]
    for x,y in loader:
        x=x.to(device); logits=model(x)
        ps.append(logits.argmax(1).cpu().numpy())
        ys.append(y.numpy())
    return np.concatenate(ys), np.concatenate(ps)

te_loss, te_acc = eval_epoch(model,test_loader,criterion)
print(f'[TEST] acc={te_acc:.4f}, loss={te_loss:.4f}')
y_true, y_pred = preds_and_labels(model,test_loader)

cm = confusion_matrix(y_true,y_pred,labels=list(range(10)))
plt.figure(figsize=(6,5))
plt.imshow(cm,interpolation='nearest')
plt.title('Confusion Matrix'); plt.colorbar()
plt.xticks(range(10),class_names,rotation=45); plt.yticks(range(10),class_names)
plt.tight_layout(); plt.xlabel('Pred'); plt.ylabel('True'); plt.show()

print(classification_report(y_true,y_pred,target_names=class_names))

In [ ]:
# 저장/로딩
torch.save(model.state_dict(),'cnn_cpu_opt.pt')
print('saved: cnn_cpu_opt.pt')
m2 = CNN().to(device); m2.load_state_dict(torch.load('cnn_cpu_opt.pt', map_location=device)); m2.eval()
print('reload ok')